In [13]:
%pip install -U cohere pinecone datasets pandas

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [14]:
import cohere

co = cohere.Client("COHERE_API_KEY")


In [28]:
import pandas as pd
df = pd.read_csv("C:/Users/Vicky/Documents/RAG Bot/dataset.csv",header=0)
df.shape

(1005, 2)

In [36]:
df.iloc[:, 0]

0                How do I take a screenshot on an iPhone?
1              How do I change my wallpaper on an iPhone?
2                How do I make a phone call on an iPhone?
3              How do I send a text message on an iPhone?
4                         How do I use Siri on an iPhone?
                              ...                        
1000    How to clear cache on Safari on my Mac for imp...
1001        What to do if my AirPods Pro have low volume?
1002    Can I use a wireless trackpad with my Mac for ...
1003    How to transfer files from my Mac to my iPhone...
1004    Is it possible to use Siri on my Mac without c...
Name: Question, Length: 1005, dtype: object

In [37]:
# Assuming your DataFrame has 'Question' and 'Answer' columns
df['combined'] = df['Question'].astype(str) + " " + df['Answer'].astype(str)

# Convert the 'combined' column to a list of strings
texts = df['combined'].tolist()

# Generate embeddings
embeds = co.embed(
    texts=texts,
    model='embed-english-v3.0',
    input_type='search_document',
    truncate='END'
).embeddings

In [38]:
import numpy as np

shape = np.array(embeds).shape
print(shape)

# [Out]:
# (1000, 1024)


(1005, 1024)


In [44]:
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
pc = Pinecone(api_key='PINECONE_API_KEY')

index_name = "quickstart"

pc.create_index(
    name=index_name,
    dimension=1024, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)


# connect to index
index = pc.Index(index_name)


In [46]:
batch_size = 128
shape = (len(embeds),)  # Get the shape based on the number of embeddings

# Create list of IDs for each embedding (assuming one embedding per row)
ids = [str(i) for i in range(shape[0])]

df['Question'] = df['Question'].fillna('')
df['Answer'] = df['Answer'].fillna('')

# Assuming `df` contains the columns 'Question' and 'Answer'
# Create list of metadata dictionaries with both question and answer
meta = [{'question': question, 'answer': answer} for question, answer in zip(df['Question'], df['Answer'])]

# Create list of (id, vector, metadata) tuples to be upserted
to_upsert = list(zip(ids, embeds, meta))

# Upsert vectors in batches
for i in range(0, shape[0], batch_size):
    i_end = min(i + batch_size, shape[0])
    index.upsert(vectors=to_upsert[i:i_end])

# View index statistics
print(index.describe_index_stats())

# Expected Output Example:
# {'dimension': 1024,
#  'index_fullness': 0.0,
#  'namespaces': {'': {'vector_count': 1000}},
#  'total_vector_count': 1000}


{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1005}},
 'total_vector_count': 1005}


In [59]:
query = "How to save photos from a browser?"

# create the query embedding
xq = co.embed(
    texts=[query],
    model='embed-english-v3.0',
    input_type='search_query',
    truncate='END'
).embeddings

print(np.array(xq).shape)

# query, returning the top 5 most similar results
res = index.query(vector=xq, top_k=5, include_metadata=True)


(1, 1024)


In [60]:
for match in res['matches']:
    print(f"Score: {match['score']:.2f}")
    print(f"Question: {match['metadata']['question']}")
    print(f"Answer: {match['metadata']['answer']}\n")


Score: 0.58
Question: How do I take advantage of the Safari web browser to save web pages for offline reading or share them with others?
Answer: To save a web page for offline reading, open the web page you want to save and tap on the bookmark icon (looks like a book) in the address bar. Choose "Add Bookmark" and select the location where you want to save it. Saved web pages can be accessed by tapping on the bookmark icon and selecting "Reading List." To share a web page, tap on the share icon (looks like an arrow pointing upwards) in the address bar. Choose the sharing method, such as email, messaging, or social media.

Score: 0.58
Question: How do I take advantage of the Safari web browser to save web pages for offline reading or share them with others?
Answer: To save a web page for offline reading, open the web page you want to save and tap on the bookmark icon (looks like a book) in the address bar. Choose "Add Bookmark" and select the location where you want to save it. Saved web

In [58]:
# Define the user query
query = "How to save photos from a browser?"

# Create the query embedding
xq = co.embed(
    texts=[query],
    model='embed-english-v3.0',
    input_type='search_query',
    truncate='END'
).embeddings

print(np.array(xq).shape)

# Query the index, returning the top 5 most similar results with metadata
res = index.query(vector=xq, top_k=5, include_metadata=True)

# Retrieve relevant context from the query result
context = ""
for match in res['matches']:
    context += f"Question: {match['metadata']['question']}\n"
    context += f"Answer: {match['metadata']['answer']}\n\n"


combined_message = f"User Query: {query}\n\nRelevant Context:\n{context}"
    
    # Generate search queries, if any        
response = co.chat(message=combined_message,
                    model="command-r-plus")

print(response.text)
print(response)


(1, 1024)
To save photos from a browser, simply follow these steps:

1. Open the web page that contains the photo you want to save.

2. Depending on your browser, right-click or long-press on the image. A menu will appear.

3. From the menu, select the "Save Image" or "Save Picture As..." option. The exact wording may vary slightly depending on your browser.

4. Choose the location on your computer where you want to save the photo. You can create a new folder or select an existing one.

5. Click the "Save" button, and the photo will be downloaded and saved to the specified location.

That's it! You have now successfully saved a photo from a browser to your computer. You can then access the photo anytime from the chosen location and even transfer it to other devices if needed.
